# Interactor Finder
The `InteractorFinder` class is used to identify specifically linked nodes to your starting case. Here, we show how one initializes this class by choosing the MAPT protein with phosphorylation modification, and finds all causally linked neighbors.

In [1]:
from drugintfinder.finder import InteractorFinder

# Initialize with base information
finder = InteractorFinder(symbol="MAPT", pmods=["pho"], edge="causal")

# Select for matching starting protein nodes (i.e. MAPT protein) and find all interactors
neighbors = finder.find_interactors(target_type="protein")
neighbors.head()

,id,pmod_type,target_bel,target_symbol,target_type,relation_type,interactor_bel,interactor_name,interactor_type,pmid,pmc,target_species
0,1,pho,"p(HGNC:""MAPT"",loc(MESHA:""Cerebrospinal Fluid"")...",MAPT,protein,increases,"p(HGNC:""TREM2"",var(""p.Arg47His""))",TREM2,protein,28768545,PMC5541421,9606
1,4,pho,"p(HGNC:""MAPT"",pmod(Ph,S,357))",MAPT,protein,increases,"act(p(HGNC:""GSK3B""),ma(kin))",None,activity,17389597,None,9606
2,5,pho,"p(HGNC:""MAPT"",pmod(Ph,S,357))",MAPT,protein,increases,"act(p(HGNC:""GSK3B""),ma(kin))",None,activity,17360711,None,9606
3,6,pho,"p(HGNC:""MAPT"",pmod(Ph,S,199))",MAPT,protein,increases,"a(MESHC:""calyculin A"")",calyculin A,abundance,21297267,None,9606
4,7,pho,"p(HGNC:""MAPT"",pmod(Ph,S,199))",MAPT,protein,increases,"act(p(HGNC:""CDK5""),ma(kin))",None,activity,12387894,None,9606


### Druggable Interactors
While knowing the neighbors of select nodes is useful, knowing which ones can be targeted by drugs and compounds is even more informative. The InteractorFinder class has a method for searching out those special neighbors. By default, these neighbors are isolated to proteins as the KG restricts drug-target interactions to those occurring between proteins and compounds.

In [2]:
druggable_ints = finder.druggable_interactors()
druggable_ints.head()

,id,drug,drugbank_id,chembl_id,pubchem_id,interactor_type,interactor_bel,interactor_name,capsule_interactor_bel,capsule_interactor_type,...,target_bel,target_symbol,target_type,relation_type,pmod_type,pmid,pmc,rel_rid,drug_rel_rid,drug_rel_actions
0,1,"N'-(Pyrrolidino[2,1-B]Isoindolin-4-On-8-Yl)-N-...",DB04186,CHEMBL141247,445840.0,protein,"p(HGNC:""CDK2"")",CDK2,None,None,...,"p(HGNC:""MAPT"",pmod(Ph,S,199))",MAPT,protein,directly_increases,pho,8282104,None,#570:10,#1898:10624,None
1,2,"1-(3,5-DICHLOROPHENYL)-5-METHYL-1H-1,2,4-TRIAZ...",DB07852,None,2763754.0,protein,"p(HGNC:""CDK2"")",CDK2,None,None,...,"p(HGNC:""MAPT"",pmod(Ph,S,199))",MAPT,protein,directly_increases,pho,8282104,None,#570:10,#1898:10917,None
2,3,N(6)-dimethylallyladenine,DB08768,CHEMBL476189,92180.0,protein,"p(HGNC:""CDK2"")",CDK2,None,None,...,"p(HGNC:""MAPT"",pmod(Ph,S,199))",MAPT,protein,directly_increases,pho,8282104,None,#570:10,#1898:11306,None
3,4,"(5E)-2-Amino-5-(2-pyridinylmethylene)-1,3-thia...",DB07529,None,46937079.0,protein,"p(HGNC:""CDK2"")",CDK2,None,None,...,"p(HGNC:""MAPT"",pmod(Ph,S,199))",MAPT,protein,directly_increases,pho,8282104,None,#570:10,#1898:11332,None
4,5,"4-{5-[(Z)-(2-IMINO-4-OXO-1,3-THIAZOLIDIN-5-YLI...",DB07534,CHEMBL233149,5729339.0,protein,"p(HGNC:""CDK2"")",CDK2,None,None,...,"p(HGNC:""MAPT"",pmod(Ph,S,199))",MAPT,protein,directly_increases,pho,8282104,None,#570:10,#1899:10523,None


# Ranker
The `Ranker` class is used to generate useful statistics about identified druggable interactors. Because this scoring is restricted to druggable interactors, only the starting node and pmods are needed to initialize.  

The ranking algorithm requires a bit of information to score everything, the first time it is used, it will need to download information on BioAssays and other resources which it will then store in a locally made SQLite database. The total space used is less than 100 MB.  

The download will take a couple of minutes during the first ranking, but subsequent rankings will be much faster.  

## Drug Focused
By default, the ranking system will provde a table that is composed of unique drug/target combinations and their associated metadata. Therefore, targets will appear in multiple rows (the number of rows is qual to the number of a associated drugs).

In [3]:
from drugintfinder.ranker import Ranker

ranker = Ranker(symbol="MAPT", pmods=["pho"], reward=1, penalty=-1)
ranker.rank()  # Performs the ranking
summary = ranker.summarize()  # Create a summary os the statistics
summary.head()

Counting edges: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 1027.94it/s]


,Drug,Target,Synergizes,Number of BioAssays for Target,Number of Causal Edges for Target,Drug Patent Ongoing,Generic Version of Drug Available,Number of Drug Targets
0,"(13R,15S)-13-METHYL-16-OXA-8,9,12,22,24-PENTAA...",CDK2,N/A,1518,15,No,No,1
1,Phosphonothreonine,MAPK12,N/A,647,6,No,No,10
2,Procyclidine,CHRM1,N/A,1115,9,No,Yes,4
3,4-[5-(Trans-4-Aminocyclohexylamino)-3-Isopropy...,CDK2,N/A,1518,15,No,No,1
4,quercetin,PKN1,N/A,108,1,No,No,N/A


## Target Focused
It is also possible to create a ranking summary that is focused on the target metadata instead. The ranking table can be changed using the `pivot` flag to collapse the results into individual target rows with BioAssay, drug, and edge counts listed.

In [4]:
from drugintfinder.ranker import Ranker

ranker = Ranker(symbol="MAPT", pmods=["pho"], reward=1, penalty=-1)
ranker.rank()  # Performs the ranking
summary = ranker.summarize(pivot=True)  # Create a summary os the statistics
summary.head()

Counting edges: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 1042.84it/s]


,Target,Number of BioAssays for Target,Number of Causal Edges for Target,Number of Known Drugs,BioAssay to Known Drug Ratio
51,HDAC6,1938,180,10,193.800000
48,ABL1,1929,10,18,107.166667
0,CDK2,1518,15,137,11.080292
24,MAPK14,1490,7,57,26.140351
27,AKT1,1259,14,11,114.454545


# Connecting to a Different Knowledge Graph
By default, this package connects to the Alzheimer's Disease based Knowledge Graph (KG) developed under the MAVO project, available at https://graphstore.scai.fraunhofer.de. There are other KGs available, however, and here you can choose to connect to a different one if desired.  

The commented out code shows how one can connect instead to the COVID KG.

In [5]:
from ebel_rest import connect
connect(user="covid_user", password="covid", db_name="covid", server="https://graphstore.scai.fraunhofer.de")